## Workflow to create PFOB ROI's algorithmically (using parcels from Julian et al 2012)

Attempt to create a workflow to create ROIs algorithmically 
[Similar to Fedorenko 2010, and Julian 2012]

This code does:
1. Take individual activation maps for a certain contrast
2. Transform the maps to a common space
3. Threshold the maps at a reasonable level and save a probabilistic map

To DO
4. Parcellate the overlap map by using an image segmentation algorithm
5. Select a set of meaningful parcels
6. Define subject specific ROI's by intersecting each meaningful parcel with 
each individual thresholded activation map
7. Define the ROI's based on this


dependencies: antsApplyTransforms

In [1]:
# %% IMPORTS and PATHS
from glob import glob
import os
from nipype.interfaces.ants import ApplyTransforms
from nibabel import load, save, Nifti1Image
from nilearn import plotting
import numpy as np
from nistats.thresholding import map_threshold
import matplotlib.pyplot as plt

/home/ilkay/miniconda3/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


# Set paths

In [2]:
root = '/home/ilkay/MPI-Documents/hack_19/data/'
ants_files = sorted(glob(root + 'fmriprep/sub-*/anat/sub-*_T1w_target-'\
                               'MNI152NLin2009cAsym_warp.h5'))
nr_sub = len(ants_files) 
mni_temp = root + 'mni152.nii'

The mni template (mni_icbm152_nlin_asym_09c.nii) is 1mm iso, but the func data is 3 mm iso.
To make them compateblecreate a 3 mm version of the mni template.

In [4]:
%%bash
bash_root='/home/ilkay/MPI-Documents/hack_19/data/'
mni_temp=${bash_root}'/mni152.nii'
mni_temp_3mm=${bash_root}'/mni152_3mm.nii'

flirt -in ${mni_temp} -ref ${mni_temp} \
-applyisoxfm 3 -interp spline -out ${mni_temp_3mm}

1. Take individual activation maps for a certain contrast 

In [6]:
# PLACE
p_zmap = sorted(glob(root + 'analysis/sub-*/pfobloc/pfobloc.feat/stats/zstat13.nii.gz'))
# FACE
f_zmap = sorted(glob(root + 'analysis/sub-*/pfobloc/pfobloc.feat/stats/zstat12.nii.gz'))
# OBJECT
o_zmap = sorted(glob(root + 'analysis/sub-*/pfobloc/pfobloc.feat/stats/zstat11.nii.gz'))
# BODY
b_zmap = sorted(glob(root + 'analysis/sub-*/pfobloc/pfobloc.feat/stats/zstat14.nii.gz'))

pfob = [p_zmap, f_zmap, o_zmap, b_zmap]
names = ['place_prob', 'face_prob', 'object_prob', 'body_prob']

2. Transform the maps to a common space 3x3x3 mni

In [ ]:
mni_temp = root + 'mni152_3mm.nii.gz'
for loc in pfob:
    for c, zmap in enumerate(loc):
        norm = ApplyTransforms(
                reference_image=mni_temp,
                input_image_type=3,
                float=True,
                interpolation='Linear',
                invert_transform_flags=[False],
                output_image=zmap.replace('.nii.gz', '_inMNI_3mm.nii.gz'),
                input_image=zmap,
                transforms=ants_files[c])
        res = norm.run()
        print(c+1, zmap)

Load the normalized maps

In [7]:
p_norm = sorted(glob(root + 'analysis/sub-*/pfobloc/pfobloc.feat/stats/zstat13_inMNI_3mm.nii.gz'))
f_norm = sorted(glob(root + 'analysis/sub-*/pfobloc/pfobloc.feat/stats/zstat12_inMNI_3mm.nii.gz'))
o_norm = sorted(glob(root + 'analysis/sub-*/pfobloc/pfobloc.feat/stats/zstat11_inMNI_3mm.nii.gz'))
b_norm = sorted(glob(root + 'analysis/sub-*/pfobloc/pfobloc.feat/stats/zstat14_inMNI_3mm.nii.gz'))
pfob_norm = [p_norm, f_norm, o_norm, b_norm]

3. Threshold and binarize the maps at a reasonable level
Save probabilistic maps and images out

In [25]:
h, thr ='fdr', 0.05 # thresholding parameters
prob_maps = root + 'prob_maps'

if not os.path.exists(prob_maps):
    os.makedirs(prob_maps)

for c, loc in enumerate(pfob_norm):
    zmaps = [load(zmap) for zmap in loc]
    thr_images, thresholds = [], []
    
    for i in range(nr_sub):
        thr_img, threshold = map_threshold(zmaps[i], level=thr, height_control=h)
        thr_images.append(thr_img)
        thresholds.append(threshold)
    
    data = [zmap.get_data() for zmap in thr_images]
    bin_data = []
    for i  in range(nr_sub):
        bin_data.append(np.where(data[i] > thresholds[i], 1, 0))

    # sum the images
    sum_img = np.sum(bin_data, axis=0)
    # save out
    out = Nifti1Image(sum_img, header=zmaps[0].header, affine=zmaps[0].affine)
    save(out, prob_maps + '/' + names[c] + "sum.nii.gz")
    
    # plots
    plotting.plot_stat_map(out, bg_img=mni_temp)
    plt.savefig(prob_maps + '/' + names[c] + '_sum_plot.png', dpi=300)
    plt.close()
    
    plotting.plot_glass_brain(out, black_bg=True, display_mode='lyrz', colorbar=True,
    title=names[c])
    plt.savefig(prob_maps + '/' + names[c] + '_sum_glass_summary.png', dpi=300)
    plt.close()
    
    
    
    # threshold the sum image and save out as a mask
    thr_sum = np.where(sum_img > 6, 1, 0)
    
    out = Nifti1Image(thr_sum, header=zmaps[0].header, affine=zmaps[0].affine)
    save(out, prob_maps + '/' + names[c] + "_thr_sum.nii.gz")
    
    # plots
    plotting.plot_stat_map(out, bg_img=mni_temp, title=names[c])
    plt.savefig(prob_maps + '/' + names[c] + '_thr_sum.png', dpi=300)
    plt.close()

    plotting.plot_glass_brain(out, black_bg=True, display_mode='lyrz', colorbar=True,
    title=names[c])
    plt.savefig(prob_maps + '/' + names[c] + '_thr_sum_summary.png', dpi=300)
    plt.close()

/home/ilkay/miniconda3/lib/python3.6/site-packages/scipy/ndimage/measurements.py:272: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return _nd_image.find_objects(input, max_label)
/home/ilkay/miniconda3/lib/python3.6/site-packages/scipy/ndimage/measurements.py:272: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return _nd_image.find_objects(input, max_label)
/home/ilkay/miniconda3/lib/python3.6/site-packages/nilearn/_utils/cache_mixin.py:232: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if (memory.cachedir is None and memory_level is not None
/home/ilkay/miniconda3/lib/python3.6/site-packages/scipy/ndimage/measurements.py:272: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  re

4. Parcellate the overlap map by using an image segmentation algorithm

In [17]:
sum_img[sum_img < 6] = 0

np.shape(sum_img)

(64, 76, 64)

transform the rois back to native space

In [7]:
ants_files_rev = sorted(glob(root + 'fmriprep/sub-*/anat/sub-*_T1w_space-MNI152NLin2009cAsym_target-T1w_warp.h5'))
ref_imgs = sorted(glob(root + 'analysis/sub-*/pfobloc/pfobloc_group/pfobloc.feat/thresh_zstat12.nii.gz'))
T1_refs = sorted(glob(root + '/fmriprep/sub-*/anat/sub-*_T1w_preproc.nii.gz'))

In [340]:
for s in range(nr_sub):
    sub = 'sub-{}'.format(str(s+1).zfill(2))
    rois = sorted(glob(root + 'analysis/' + sub + '/pfobloc/pfobloc_rois/*_thr_0.01.nii.gz'))
    for roi in rois:
        norm = ApplyTransforms(
                reference_image=ref_imgs[s],
                dimension=3,
                input_image_type=3,
                float=True,
                interpolation='NearestNeighbor',
                invert_transform_flags=[False],
                output_image=roi.replace('.nii.gz', '_inT1.nii.gz'),
                input_image=roi,
                transforms=ants_files_rev[s])
        res = norm.run()

In [ ]:
# Transform maps back to subject's T1 space

In [8]:
for s in range(nr_sub):
    sub = 'sub-{}'.format(str(s+1).zfill(2))
    rois = sorted(glob(root + 'analysis/' + sub + '/pfobloc/pfobloc_rois/*_thr_0.05.nii.gz'))
    for roi in rois:
        norm = ApplyTransforms(
                reference_image=ref_imgs[s],
                dimension=3,
                input_image_type=3,
                float=True,
                interpolation='NearestNeighbor',
                invert_transform_flags=[False],
                output_image=roi.replace('.nii.gz', '_inT1.nii.gz'),
                input_image=roi,
                transforms=ants_files_rev[s])
        res = norm.run()